# Statistical Concepts

## Introduction

Say you work for a major social media website. Your boss always says "data drives all our decisions" and it seems to be true. Metrics are collected on all users of your website, terabytes of data stored in replicated databases.

One day, your boss wants to know if college students are engaging in your website. You pull up the records for users in that age bracket and look at them one by one. The first person only spent half a second on your website before closing the tab — that does not look good. But the second person was on the site for thirty minutes! That is a running average of 15 minutes site time per user, but you still have half a million records to look at.

On top of that, you need to compare it against other age brackets (and the average overall). That is going to take a lot of time if you do it all by hand, and you are still not sure what your methodology for proving college students spend enough time on your website to be "engaged".

When conducting data analysis, we want to say something meaningful about our data. Often, we want to know if a change or difference we see in a dataset is "real" or if it is just a normal fluctuation or a result of the specific sample of people we have chosen to measure. A difference we observe in our data is only important if we can be reasonably sure that it is representative of the population as a whole, and reasonably sure that our result is repeatable.

This question of whether a difference is significant or not is essential to making decisions based on that difference. Some instances where this might come up include:

* Performing an A/B test — are the different observations really the results of different conditions (i.e., Condition A vs. Condition B)? Or just the result of random chance?
* Conducting a survey — is the fact that men gave slightly different responses than women a real difference between men and women? Or just the result of chance?

In this lesson, we will cover the fundamental concepts that will help us create tests to measure our confidence in our statistical results:

* Sample means and population means
* The Central Limit Theorem
* Why we use hypothesis tests
* What errors we can come across and how to classify them

***
### Pseudo Code Example

> ## Are the Millennials Engaged?

> You work at the global megacorp social network SpyPy. SpyPy has 1.5 billion daily users, and you want to make sure that people in the millennial age bracket are engaging with your website. Your boss seems particularly frazzled by this question, and he is put it on you to find out. You decide that "engagement" means spending more than the average of seven minutes on the website. You fire up your data-science stack in Python and first check the average time -- which turns out to be near 11 whole minutes! But you cannot really tell if they are really spending more time or if it is just random chance that a few of your users left the browser open and walked away. You write the following code:

>       import spypy
>       from scipy.stats import ttest_1samp
>       
>       millennial_times = spypy.get_site_times_for_demographic('millennial')
>       t_stat, p_val = ttest_1samp(millennial_times, 7)
>       
>       if p_val < .05:
>          print "The Millennials are engaged!"
>       else:
>           print "The Millennials are not engaged :(!"
>\>\> The Millennials are engaged!

> ## SpyPy: We're Significantly Different

> Well that is great news! Millennials are, for the most part, spending around 10 minutes on your website. But before you break out the champagne glasses your boss is in a frenzy again, this time about Metropolitan Statistical Areas (MSAs). You are tasked with finding if people in cooler climates post more pictures on SpyPy than people in warmer climates. You cross corroborate with weather data and run a statistical test on the info.

>       from scipy.stats import ttest_ind
>       
>       warmer_weather_picture_count = spypy.get_number_pictures_for_climate('hot')
>       colder_weather_picture_count = spypy.get_number_pictures_for_climate('cold')
>       
>       t_stat, p_val = ttest_ind(warmer_weather_picture_count, colder_weather_picture_count)
>       
>       if p_val < .05:
>           print "People from colder climates post a different number of pictures compared to people from warmer climates"
>       else:
>           print "Climate doesn't appear to affect the number of pictures posted"
>\>\> Climate doesn't appear to affect the number of pictures posted

> ##  SpyPy: Because We Care About Your Data

> Seems like climate probably does not really affect the number of times people post pictures. Not really sure why that would have been the case anyway. SpyPy has a new feature that you think will get people to interact with the website for longer: SpyPy Stories. It is preliminarily being launched to 8 million users and the internal goal is to get 2 million people to post SpyPy Stories in the first week. Unfortunately, only 1,997,893 people posted SpyPy Stories this week. We want to know if this is a significant difference from our goal -- did we pretty much meet it or did we seriously miss? You know how to answer this question:

>       from scipy.stats import binom_test
>       
>       number_of_trials = 8000000
>       expected_successes = 2000000
>       actual_successes = 1997893
>       expected_success_rate = float(expected_successes) / float(number_of_trials)
>       
>       p_val = binom_test(actual_successes, n=number_of_trials, p=expected_success_rate)
>       if p_val < 0.05:
>           print "We didn't hit our target by a significant amount"
>       else:
>           print "We just missed our target by a very small amount!"
>\>\> We just missed our target by a very small amount!

> Looks like we came very close to hitting our target for SpyPy Stories! You have saved the day so many times already! Your boss comes by to thank you for all the hard work you put in today and says you have made significant contributions to the team. You tell her you are not sure if that's true, but you definitely have a way of finding out.

***

## Sample Mean and Population Mean

Suppose you want to know the average height of an oak tree in your local park. On Monday, you measure 10 trees and get an average height of 32 ft. On Tuesday, you measure 12 different trees and reach an average height of 35 ft. On Wednesday, you measure the remaining 11 trees in the park, whose average height is 31 ft. Overall, the average height for all trees in your local park is 32.8 ft.

The individual measurements on Monday, Tuesday, and Wednesday are called samples. A _sample_ is a subset of the entire population. The mean of each sample is the _sample mean_ and it is an estimate of the _population mean_.

Note that the sample means (32 ft., 35 ft., and 31 ft.) were all close to the population mean (32.8 ft.), but were all slightly different from the population mean and from each other.

For a population, the mean is a constant value no matter how many times it is recalculated. But with a set of samples, the mean will depend on exactly what samples we happened to choose. From a sample mean, we can then extrapolate the mean of the population as a whole. There are many reasons we might use sampling, such as:

* We do not have data for the whole population.
* We have the whole population data, but it is so large that it is infeasible to analyze.
* We can provide meaningful answers to questions faster with sampling.

When we have a numerical dataset and want to know the average value, we calculate the mean. For a population, the mean is a constant value no matter how many times it us recalculated. But with a set of samples, the mean will depend on exactly what samples we happened to choose. From a sample mean, we can then extrapolate the mean of the population as a whole.

***
### Exercise

1. In the cell below, we have generated random samples from a population with a given population mean. We are going to look at how the means of different samples can vary within the same population.

    Run the code to print out the population mean and the mean of `sample_1`.

In [1]:
import numpy as np

population = np.random.normal(loc=65, scale=3.5, size=300)
population_mean = np.mean(population)

print("Population Mean: {}".format(population_mean))

sample_1 = np.random.choice(population, size=30, replace=False)
sample_2 = np.random.choice(population, size=30, replace=False)
sample_3 = np.random.choice(population, size=30, replace=False)
sample_4 = np.random.choice(population, size=30, replace=False)
sample_5 = np.random.choice(population, size=30, replace=False)

sample_1_mean = np.mean(sample_1)
print("Sample 1 Mean: {}".format(sample_1_mean))

sample_2_mean = "Not calculated"
sample_3_mean = "Not calculated"
sample_4_mean = "Not calculated"
sample_5_mean = "Not calculated"

print("Sample 2 Mean: {}".format(sample_2_mean))
print("Sample 3 Mean: {}".format(sample_3_mean))
print("Sample 4 Mean: {}".format(sample_4_mean))
print("Sample 5 Mean: {}".format(sample_5_mean))

Population Mean: 64.8765469597968
Sample 1 Mean: 64.62766524290473
Sample 2 Mean: Not calculated
Sample 3 Mean: Not calculated
Sample 4 Mean: Not calculated
Sample 5 Mean: Not calculated


2. Replace the `"Not calculated"` strings with calculations of the means for `sample_2`, `sample_3`, `sample_4`, and `sample_5`.

In [2]:
import numpy as np

population = np.random.normal(loc=65, scale=3.5, size=300)
population_mean = np.mean(population)

print("Population Mean: {}".format(population_mean))

sample_1 = np.random.choice(population, size=30, replace=False)
sample_2 = np.random.choice(population, size=30, replace=False)
sample_3 = np.random.choice(population, size=30, replace=False)
sample_4 = np.random.choice(population, size=30, replace=False)
sample_5 = np.random.choice(population, size=30, replace=False)

sample_1_mean = np.mean(sample_1)
print("Sample 1 Mean: {}".format(sample_1_mean))

sample_2_mean = np.mean(sample_2)
sample_3_mean = np.mean(sample_3)
sample_4_mean = np.mean(sample_4)
sample_5_mean = np.mean(sample_5)

print("Sample 2 Mean: {}".format(sample_2_mean))
print("Sample 3 Mean: {}".format(sample_3_mean))
print("Sample 4 Mean: {}".format(sample_4_mean))
print("Sample 5 Mean: {}".format(sample_5_mean))

Population Mean: 64.95081878834704
Sample 1 Mean: 65.15158174520336
Sample 2 Mean: 64.00229348880796
Sample 3 Mean: 64.33115465238694
Sample 4 Mean: 65.07707386198688
Sample 5 Mean: 64.70478142898136


3. Look at the population mean and the sample means. Are they all the same? All different? Why?

> The sample means vary slightly as these represent only a fraction of the population. Each sample has different values selected and therefore they vary. In very rare cases the samples might match the population or other samples.

***

Central Limit Theorem

Perhaps, this time, you are a tailor of school uniforms at a middle school. You need to know the average height of people from 10-13 years old in order to know which sizes to make the uniforms. Knowing the best decisions are based on data, you set out to do some research at your local middle school.

Organizing with the school, you measure the heights of some students. Their average height is 145 centimeters. You know a little about sampling and decide that measuring 30 out of the 300 students gives enough data to assume 145 cm is roughly the average height of everyone at the middle school. You set to work with this dimension and make uniforms that fit people of this height, some smaller and some larger.

Unfortunately, when you go about making your uniforms many reports come back saying that they are too small. Something must have gone wrong with your decision-making process! You go back to collect the rest of the data: you measure the sixth graders one day (144, not so bad), the seventh graders after that (152 centimeters tall on average), and the eighth graders the next day (163 centimeters!). Your sample mean was so far off from your population mean. How did this happen?

Well, your sample selection was skewed to one direction of the total population. It looks like you must have measured more sixth graders than is representative of the whole middle school. How do you get an average sample height that looks more like the average population height?

In the previous exercise, we looked at different sets of samples taken from a population and how the mean of each set could be different from the population mean. This is a natural consequence of the fact that a set of samples has less data than the population to which it belongs. If our sample selection is poor then we will have a sample mean seriously skewed from our population mean.

In order to quantify our uncertainty about whether a particular sample mean is similar to the population mean, consider the following thought experiment: suppose we go back and take 100 more samples of 30 students each. For each, we record the **mean** height of the students in the sample. When we are done, we will have 100 different sample means, each from a different randomly chosen group of 30 students! Now, we can look at those 100 sample means and see how much they differ from each other. If our sample means are all between 151 and 153 centimeters, we will probably feel more confident that we can trust any individual sample mean to be representative of the population; however, if our sample means range from 142 to 168 centimeters, we will be less confident about any individual estimate.

In real life, we probably will not be able to collect lots of repeated samples, but luckily we can use something called the _Central Limit Theorem_, which tells us that a sample mean is more likely to be similar to the population mean if the sample size is large.

***

### Exercise

1. In the cell below we have generated increasingly larger sets of samples from the same population. Calculate the mean of each set.

In [3]:
# Create population and find population mean
population = np.random.normal(loc=65, scale=100, size=3000)
population_mean = np.mean(population)

# Select increasingly larger samples
extra_small_sample = population[:10]
small_sample = population[:50]
medium_sample = population[:100]
large_sample = population[:500]
extra_large_sample = population[:1000]

In [4]:
# Calculate the mean of those samples
extra_small_sample_mean = np.mean(extra_small_sample)
small_sample_mean = np.mean(small_sample)
medium_sample_mean = np.mean(medium_sample)
large_sample_mean = np.mean(large_sample)
extra_large_sample_mean = np.mean(extra_large_sample)

2. Take a look at the means you have calculated. How does the relationship these values have with the population mean change as the sample set gets larger?

In [5]:
# Print them all out!
print("Extra Small Sample Mean: {}".format(extra_small_sample_mean))
print("Small Sample Mean: {}".format(small_sample_mean))
print("Medium Sample Mean: {}".format(medium_sample_mean))
print("Large Sample Mean: {}".format(large_sample_mean))
print("Extra Large Sample Mean: {}".format(extra_large_sample_mean))

print("\nPopulation Mean: {}".format(population_mean))

Extra Small Sample Mean: 80.12101108458913
Small Sample Mean: 54.49060441902503
Medium Sample Mean: 74.3839861652025
Large Sample Mean: 66.7768353328006
Extra Large Sample Mean: 64.29931846392815

Population Mean: 65.5577289726518


> The bigger the sample the closer it is to the population mean.

***

## Type I Or Type II

When we rely on automated processes to make our decisions for us, we need to be aware of how this automation can lead to mistakes. Computer programs are as fallible as the humans who design them. As humans capable of programming, the responsibility is on us to understand what can go wrong and what we can do to contain these foreseeable problems.

In statistical hypothesis testing, we concern ourselves primarily with two types of error. The first kind of error, known as a Type I error, is finding a correlation between things that are not related. This error is sometimes called a "false positive" and occurs when the null hypothesis is rejected even though it is true.

For example, let us say you conduct an A/B test for an online store and conclude that interface B is significantly better than interface A at directing traffic to a checkout page. You have rejected the null hypothesis that there is no difference between the two interfaces. If, in reality, your results were due to the groups you happened to pick, and there is actually no significant difference between interface A and interface B in the greater population, you have been the victim of a false positive.

The second kind of error, a Type II error, is failing to find a correlation between things that are actually related. This error is referred to as a "false negative" and occurs when the null hypothesis is accepted even though it is false.

For example, with the A/B test situation, let zs say that after the test, you concluded that there was no significant difference between interface A and interface B. If there actually is a difference in the population as a whole, your test has resulted in a false negative.

***
### Exercise

1. In the cell below you will find four lists: `actual_positive`, `actual_negative`, `experimental_positive`, and `experimental_negative`. These lists represent outcomes from a statistical experiment. Also given is an `intersect` function that takes two lists as arguments and returns a list of all items in both lists.

    Examine these lists and make sure you understand what they represent.

In [6]:
def intersect(list1, list2):
  return [sample for sample in list1 if sample in list2]

# the true positives and negatives:
actual_positive = [2, 5, 6, 7, 8, 10, 18, 21, 24, 25, 29, 30, 32, 33, 38, 39, 42, 44, 45, 47]
actual_negative = [1, 3, 4, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 22, 23, 26, 27, 28, 31, 34, 35, 36, 37, 40, 41, 43, 46, 48, 49]

# the positives and negatives we determine by running the experiment:
experimental_positive = [2, 4, 5, 7, 8, 9, 10, 11, 13, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 27, 28, 32, 35, 36, 38, 39, 40, 45, 46, 49]
experimental_negative = [1, 3, 6, 12, 14, 23, 25, 29, 30, 31, 33, 34, 37, 41, 42, 43, 44, 47, 48]

2. Use the `intersect` function and the lists provided to define `type_i_errors`. This list should contain the false positives of the experiment.

In [7]:
type_i_errors = intersect(actual_negative, experimental_positive)

3. Now, define `type_ii_errors`, the list representing the false negatives of the experiment.

In [8]:
type_ii_errors = intersect(actual_positive, experimental_negative)

***

## P-Values

We have discussed how a hypothesis test is used to determine the validity of a null hypothesis. A hypothesis test provides a numerical answer, called a _p-value_, that helps us decide how confident we can be in the result. In this context, a p-value is the probability that we yield the observed statistics under the assumption that the null hypothesis is true.

A p-value of 0.05 means that if the null hypothesis is true, there is a 5% chance that an observed sample statistic could have occurred due to random sampling error. For example, in comparing two sample means, a p-value of 0.05 indicates there is a 5% chance that the observed difference in sample means occurred by random chance, even though the population means are equal.

Before conducting a hypothesis test, we determine the necessary threshold we would need before concluding that the results are significant. A higher threshold is more likely to give a _false positive_ so if we want to be very sure that the result is not due to just chance, we will select a very small threshold.

It is important that we choose the significance level before we perform our statistical hypothesis tests to yield a p-value. If we wait until after we see the results, we might pick our threshold such that we get the result we want to see. For instance, if we’re trying to publish our results, we might set a significance level that makes our results seem statistically significant. Choosing our significance level in advance helps keep us honest.

Generally, we want a p-value of less than 0.05, meaning that there is less than a 5% chance that our results are due to random chance.

****

### Exercise

1. Fill in the body of the given function `reject_null_hypothesis` to return `True` if the p-value is small enough to reject the null hypothesis (i.e., it is less than `0.05`), and return `False` otherwise.

        def reject_null_hypothesis(p_value):
        """
        Returns the truthiness of whether the null hypothesis can be rejected
  
        Takes a p-value as its input and assumes p <= 0.05 is significant
        """
        return # TO-DO
  
        hypothesis_tests = [0.1, 0.009, 0.051, 0.012, 0.37, 0.6, 0.11, 0.025, 0.0499, 0.0001]
  
        for p_value in hypothesis_tests:
            reject_null_hypothesis(p_value)

In [9]:
def reject_null_hypothesis(p_value):
  """
  Returns the truthiness of whether the null hypothesis can be rejected

  Takes a p-value as its input and assumes p <= 0.05 is significant
  """
  return p_value <= 0.05

hypothesis_tests = [0.1, 0.009, 0.051, 0.012, 0.37, 0.6, 0.11, 0.025, 0.0499, 0.0001]

for p_value in hypothesis_tests:
    print(reject_null_hypothesis(p_value))

False
True
False
True
False
False
False
True
True
True
